In [12]:
import pandas as pd
times = pd.read_csv('nytimes_articles.csv')
guardian = pd.read_csv('theGuardian_articles(FIN).csv')
aljazeera = pd.read_csv('article_links_updated.csv')

In [15]:
guardian.shape

(345, 4)

In [6]:
times.head()

,Unnamed: 0,links,articles,date
0,1230,https://www.nytimes.com/2022/03/01/us/politics...,The president made it clear in a State of the ...,2022/03/01
1,168,https://www.nytimes.com/2023/10/16/us/politics...,Supported by A presidential trip to Israel at...,2023/10/16
2,169,https://www.nytimes.com/2023/10/17/us/politics...,Supported by The explosion at a hospital high...,2023/10/17
3,170,https://www.nytimes.com/2023/10/17/style/israe...,Supported by Fashion students and professors ...,2023/10/17
4,171,https://www.nytimes.com/2023/10/18/briefing/ga...,Newsletter The president is visiting the coun...,2023/10/18


In [7]:
guardian.head()

,title,date,url,full text
0,‘People are fearful of what’s to come’: Gaza c...,2023-10-08,https://www.theguardian.com/world/2023/oct/08/...,Palestinians seek refuge but electricity and g...
1,Hamas has taken a risk with its largest ever m...,2023-10-08,https://www.theguardian.com/commentisfree/2023...,The guerrilla movement that rules Gaza may be ...
2,Monday briefing: Death toll rises to more than...,2023-10-09,https://www.theguardian.com/world/2023/oct/09/...,In today’s newsletter: Thousands of Israelis a...
3,Israel declares siege of Gaza as Hamas threate...,2023-10-09,https://www.theguardian.com/world/2023/oct/09/...,Hamas says Israeli strikes have killed some ca...
4,Hundreds of bodies recovered from Israel music...,2023-10-09,https://www.theguardian.com/world/live/2023/oc...,This blog is now closed. Follow our latest upd...


In [8]:
aljazeera.head()

,0,article,date,text_up
0,https://www.aljazeera.com/news/2024/4/24/uncov...,"[<p class=""article__subhead css-1wt8oh6""><em>M...",2024-04-24,[ More than 300 bodies retrieved from two of t...
1,https://www.aljazeera.com/news/2024/4/23/hezbo...,"[<p class=""article__subhead css-1wt8oh6""><em>T...",2024-04-23,[ The attack comes after Hezbollah said Israel...
2,https://www.aljazeera.com/news/2024/4/23/by-th...,"[<p class=""article__subhead css-1wt8oh6""><em>M...",2024-04-23,"[ More than 34,000 Palestinians have been kill..."
3,https://www.aljazeera.com/news/2024/4/23/as-mo...,"[<p class=""article__subhead css-1wt8oh6""><em>U...",2024-04-23,[ UN human rights chief calls for independent ...
4,https://www.aljazeera.com/news/2024/4/23/what-...,"[<p class=""article__subhead css-1wt8oh6""><em>T...",2024-04-23,[ Tech workers are protesting against the use ...


In [9]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

def preprocessing(corpus):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    corpus = re.sub(r'[^\w\s]', '', corpus)
    tokens = word_tokenize(corpus)
    tokens = [word.lower() for word in tokens]
    tokens = [word.replace(',' , '') for word in tokens if word not in stop_words]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

In [10]:
times['processed'] = [preprocessing(str(document)) for document in times['articles'].values]
guardian['processed'] = [preprocessing(str(document)) for document in guardian['full text'].values]
aljazeera['processed'] = [preprocessing(str(document)) for document in aljazeera['text_up'].values]

In [11]:
from gensim.models import Word2Vec
from gensim.models.word2vec import FAST_VERSION
from multiprocessing import cpu_count

model_list = []
# initialize model
for corpus_processed in [times['processed'], guardian['processed'], aljazeera['processed']]:
    w2v_model = Word2Vec(vector_size=200, # vector size
                        window=40, # window for sampling
                        sample=0.01, # subsampling rate
                        epochs=200, # iterations
                        negative=10, # negative samples
                        min_count=10, # minimum threshold
                        workers=cpu_count(), # parallelize to all cores
                        hs=0, # no hierarchical softmax when negative sampling is active
    )

    # build the vocabulary
    w2v_model.build_vocab(corpus_processed)

    # train the model
    w2v_model.train(corpus_processed,
                    total_examples=w2v_model.corpus_count,
                    epochs=w2v_model.epochs)
    model_list.append(w2v_model)

#create a dict with the corpus and the model
corpus_model_dict = {'nytimes': model_list[0], 'guardian': model_list[1], 'aljazeera': model_list[2]}

# save the models
for name, model in corpus_model_dict.items():
    model.save(f'{name}_word2vec.model')